In [1]:
from sympy import *
import numpy as np
from sympy.printing.mathml import mathml
init_printing(use_unicode=True) # allow LaTeX printing
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

$\newcommand{\imu}{\mathrm{i}}$
$\newcommand{\transp}{\mathrm{T}}$


In [2]:
a_star, a, sig_23_star, sig_31_star, sig_21_star, sig_33, sig_22, sig_11, sig_21, sig_31, sig_23 = symbols('a^*, a, \\sigma_{23}^*, \\sigma_{31\'}^*, \\sigma_{21\'}^*, \\sigma_{33}, \\sigma_{22}, \\sigma_{1\'1\'}, \\sigma_{21\'}, \\sigma_{31\'}, \\sigma_{23}', complex=true)
A = MatrixSymbol('A', 11, 1)
A_full = Matrix([a_star, a, sig_23_star, sig_31_star, sig_21_star, sig_33, sig_22, sig_11, sig_21, sig_31, sig_23])
display(Eq(A, A_full))

    ⎡     a__*      ⎤
    ⎢               ⎥
    ⎢       a       ⎥
    ⎢               ⎥
    ⎢\sigma_{23}__* ⎥
    ⎢               ⎥
    ⎢\sigma_{31'}__*⎥
    ⎢               ⎥
    ⎢\sigma_{21'}__*⎥
    ⎢               ⎥
A = ⎢  \sigma_{33}  ⎥
    ⎢               ⎥
    ⎢  \sigma_{22}  ⎥
    ⎢               ⎥
    ⎢ \sigma_{1'1'} ⎥
    ⎢               ⎥
    ⎢ \sigma_{21'}  ⎥
    ⎢               ⎥
    ⎢ \sigma_{31'}  ⎥
    ⎢               ⎥
    ⎣  \sigma_{23}  ⎦

Division of the diffusion matrix into submatrices ${{D}_\nu}\left(\vec{A}_\nu,t\right)$ and the corresponding c-number subvectors $\vec{A}_\nu$ and noise submatrices ${{B}_\nu}\left(\vec{A}_\nu,t\right)$.


Submatrix ${{B}_\nu}\left(\vec{A}_\nu,t\right) =
    \begin{bmatrix}
      a  & -\imu a \\
      -b & -\imu c \\
      b  & \imu c  \\
      a  & \imu a  \\
    \end{bmatrix}$ 



In [3]:
# Define submatrix B_\\{1i}
# Define symbols for a,b,c and d
a1i, b1i, c1i, d1i = symbols('a, b, c, d', complex=True)
B1 = Matrix([[a1i, -I*a1i], [-b1i, -I*c1i], [b1i, I*c1i], [d1i, I*d1i]])
B1_real = B1.subs(d1i, a1i)
B1_complex = B1.subs(d1i, conjugate(a1i))
Bnu = MatrixSymbol('B_\\nu', 4, 2)
Eq_Bnu = Eq(Bnu, B1_complex)
display(Eq_Bnu)
# Calc submatrix D_\\nu
D1 = B1_complex*B1_complex.T
Dnu = MatrixSymbol('D_\\nu', 4, 4)
Eq_Dnu = Eq(Dnu, D1)
display(Eq_Dnu)

        ⎡a   -ⅈ⋅a⎤
        ⎢        ⎥
        ⎢-b  -ⅈ⋅c⎥
        ⎢        ⎥
B_\nu = ⎢b   ⅈ⋅c ⎥
        ⎢        ⎥
        ⎢_     _ ⎥
        ⎣a   ⅈ⋅a ⎦

        ⎡                                           _   ⎤
        ⎢    0       -a⋅b - a⋅c   a⋅b + a⋅c     2⋅a⋅a   ⎥
        ⎢                                               ⎥
        ⎢               2    2       2    2      _     _⎥
        ⎢-a⋅b - a⋅c    b  - c     - b  + c   - b⋅a + c⋅a⎥
D_\nu = ⎢                                               ⎥
        ⎢                2    2     2    2      _     _ ⎥
        ⎢a⋅b + a⋅c    - b  + c     b  - c     b⋅a - c⋅a ⎥
        ⎢                                               ⎥
        ⎢      _         _     _    _     _             ⎥
        ⎣  2⋅a⋅a     - b⋅a + c⋅a  b⋅a - c⋅a       0     ⎦

In [4]:
# Substitute submatrix B_\\nu
display(Matrix([sig_23_star, sig_33, sig_22, sig_23]))
r32 = Symbol('r_{32}', real=true)
a11 = sqrt(r32)
b11 = a11 * (sig_23_star + sig_23) / 2
c11 = a11 * (sig_23_star - sig_23) /2
B11 = B1_real.subs(a1i, a11).subs(b1i, b11).subs(c1i,c11)
display(Eq(MatrixSymbol('B_11', 4, 2), B11))
D11 = simplify(B11*B11.T)
display(Eq(MatrixSymbol('D_11', 4, 4), D11))
# Define full B matrix
B1i_full = zeros(11,2)
B1i_full[2,:] = B11[0,:]
B1i_full[10,:] = B11[3,:]
B1i_full[5:7,:] = B11[1:3,:]
B_full = B1i_full


⎡\sigma_{23}__*⎤
⎢              ⎥
⎢ \sigma_{33}  ⎥
⎢              ⎥
⎢ \sigma_{22}  ⎥
⎢              ⎥
⎣ \sigma_{23}  ⎦

      ⎡                  ________                                        _____
      ⎢                ╲╱ r_{32}                                    -ⅈ⋅╲╱ r_{3
      ⎢                                                                       
      ⎢   ________                                       ________             
      ⎢-╲╱ r_{32} ⋅(\sigma_{23} + \sigma_{23}__*)   -ⅈ⋅╲╱ r_{32} ⋅(-\sigma_{23
      ⎢───────────────────────────────────────────  ──────────────────────────
      ⎢                     2                                             2   
B₁₁ = ⎢                                                                       
      ⎢   ________                                       ________             
      ⎢ ╲╱ r_{32} ⋅(\sigma_{23} + \sigma_{23}__*)    ⅈ⋅╲╱ r_{32} ⋅(-\sigma_{23
      ⎢ ─────────────────────────────────────────    ─────────────────────────
      ⎢                     2                                             2   
      ⎢                                             

      ⎡          0                   -\sigma_{23}__*⋅r_{32}              \sigm
      ⎢                                                                       
      ⎢-\sigma_{23}__*⋅r_{32}  \sigma_{23}⋅\sigma_{23}__*⋅r_{32}   -\sigma_{23
D₁₁ = ⎢                                                                       
      ⎢\sigma_{23}__*⋅r_{32}   -\sigma_{23}⋅\sigma_{23}__*⋅r_{32}  \sigma_{23}
      ⎢                                                                       
      ⎣       2⋅r_{32}                -\sigma_{23}⋅r_{32}                  \si

a_{23}__*⋅r_{32}              2⋅r_{32}      ⎤
                                            ⎥
}⋅\sigma_{23}__*⋅r_{32}  -\sigma_{23}⋅r_{32}⎥
                                            ⎥
⋅\sigma_{23}__*⋅r_{32}   \sigma_{23}⋅r_{32} ⎥
                                            ⎥
gma_{23}⋅r_{32}                   0         ⎦

In [5]:
# Substitute submatrix B_\\nu
display(Matrix([sig_31_star, sig_33, sig_11, sig_31]))
r13 = Symbol('r_{1\'3}', real=true)
a12 = sqrt(r13)
b12 =  - a12 * (sig_31_star + sig_31) / 2
c12 = - a12 * (sig_31_star - sig_31) /2
B12 = B1_real.subs(a1i, a12).subs(b1i, b12).subs(c1i,c12)
display(Eq(MatrixSymbol('B_12', 4, 2), B12))
D12 = simplify(B12*B12.T)
display(Eq(MatrixSymbol('D_12', 4, 4), D12))
# Define full B matrix
B12_full = zeros(11,2)
B12_full[3,:] = B12[0,:]
B12_full[9,:] = B12[3,:]
B12_full[5,:] = B12[1,:]
B12_full[7,:] = B12[2,:]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B12_full)

⎡\sigma_{31'}__*⎤
⎢               ⎥
⎢  \sigma_{33}  ⎥
⎢               ⎥
⎢ \sigma_{1'1'} ⎥
⎢               ⎥
⎣ \sigma_{31'}  ⎦

      ⎡                   _________                                          _
      ⎢                 ╲╱ r_{1'3}                                      -ⅈ⋅╲╱ 
      ⎢                                                                       
      ⎢   _________                                         _________         
      ⎢ ╲╱ r_{1'3} ⋅(\sigma_{31'} + \sigma_{31'}__*)    ⅈ⋅╲╱ r_{1'3} ⋅(-\sigma
      ⎢ ────────────────────────────────────────────    ──────────────────────
      ⎢                      2                                                
B₁₂ = ⎢                                                                       
      ⎢   _________                                         _________         
      ⎢-╲╱ r_{1'3} ⋅(\sigma_{31'} + \sigma_{31'}__*)   -ⅈ⋅╲╱ r_{1'3} ⋅(-\sigma
      ⎢──────────────────────────────────────────────  ───────────────────────
      ⎢                      2                                                
      ⎢                                             

      ⎡           0                     \sigma_{31'}__*⋅r_{1'3}               
      ⎢                                                                       
      ⎢\sigma_{31'}__*⋅r_{1'3}   \sigma_{31'}⋅\sigma_{31'}__*⋅r_{1'3}   -\sigm
D₁₂ = ⎢                                                                       
      ⎢-\sigma_{31'}__*⋅r_{1'3}  -\sigma_{31'}⋅\sigma_{31'}__*⋅r_{1'3}  \sigma
      ⎢                                                                       
      ⎣       2⋅r_{1'3}                  \sigma_{31'}⋅r_{1'3}                 

-\sigma_{31'}__*⋅r_{1'3}               2⋅r_{1'3}      ⎤
                                                      ⎥
a_{31'}⋅\sigma_{31'}__*⋅r_{1'3}  \sigma_{31'}⋅r_{1'3} ⎥
                                                      ⎥
_{31'}⋅\sigma_{31'}__*⋅r_{1'3}   -\sigma_{31'}⋅r_{1'3}⎥
                                                      ⎥
  -\sigma_{31'}⋅r_{1'3}                    0          ⎦

In [6]:
# Substitute submatrix B_\\nu
display(Matrix([sig_21_star, sig_22, sig_11, sig_21]))
r12 = Symbol('r_{1\'2}', real=true)
a13 = sqrt(r12)
b13 =  - a13 * (sig_21_star + sig_21) / 2
c13 = - a13 * (sig_21_star - sig_21) /2
B13 = B1_real.subs(a1i, a13).subs(b1i, b13).subs(c1i,c13)
display(Eq(MatrixSymbol('B_13', 4, 2), B13))
D13 = simplify(B13*B13.T)
display(Eq(MatrixSymbol('D_13', 4, 4), D13))
# Define full B matrix
B13_full = zeros(11,2)
B13_full[4,:] = B13[0,:]
B13_full[8,:] = B13[3,:]
B13_full[6,:] = B13[1,:]
B13_full[7,:] = B13[2,:]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B13_full)

⎡\sigma_{21'}__*⎤
⎢               ⎥
⎢  \sigma_{22}  ⎥
⎢               ⎥
⎢ \sigma_{1'1'} ⎥
⎢               ⎥
⎣ \sigma_{21'}  ⎦

      ⎡                   _________                                          _
      ⎢                 ╲╱ r_{1'2}                                      -ⅈ⋅╲╱ 
      ⎢                                                                       
      ⎢   _________                                         _________         
      ⎢ ╲╱ r_{1'2} ⋅(\sigma_{21'} + \sigma_{21'}__*)    ⅈ⋅╲╱ r_{1'2} ⋅(-\sigma
      ⎢ ────────────────────────────────────────────    ──────────────────────
      ⎢                      2                                                
B₁₃ = ⎢                                                                       
      ⎢   _________                                         _________         
      ⎢-╲╱ r_{1'2} ⋅(\sigma_{21'} + \sigma_{21'}__*)   -ⅈ⋅╲╱ r_{1'2} ⋅(-\sigma
      ⎢──────────────────────────────────────────────  ───────────────────────
      ⎢                      2                                                
      ⎢                                             

      ⎡           0                     \sigma_{21'}__*⋅r_{1'2}               
      ⎢                                                                       
      ⎢\sigma_{21'}__*⋅r_{1'2}   \sigma_{21'}⋅\sigma_{21'}__*⋅r_{1'2}   -\sigm
D₁₃ = ⎢                                                                       
      ⎢-\sigma_{21'}__*⋅r_{1'2}  -\sigma_{21'}⋅\sigma_{21'}__*⋅r_{1'2}  \sigma
      ⎢                                                                       
      ⎣       2⋅r_{1'2}                  \sigma_{21'}⋅r_{1'2}                 

-\sigma_{21'}__*⋅r_{1'2}               2⋅r_{1'2}      ⎤
                                                      ⎥
a_{21'}⋅\sigma_{21'}__*⋅r_{1'2}  \sigma_{21'}⋅r_{1'2} ⎥
                                                      ⎥
_{21'}⋅\sigma_{21'}__*⋅r_{1'2}   -\sigma_{21'}⋅r_{1'2}⎥
                                                      ⎥
  -\sigma_{21'}⋅r_{1'2}                    0          ⎦

In [7]:
# Substitute submatrix B_\\nu
display(Matrix([sig_23_star, sig_22, sig_11, sig_23]))
a14 = sqrt(r12)
b14 = - a14 * (sig_23_star + sig_23) / 2
c14 = - a14 * (sig_23_star - sig_23) /2
B14 = B1_real.subs(a1i, a14).subs(b1i, b14).subs(c1i,c14)
display(Eq(MatrixSymbol('B_14', 4, 2), B14))
D14 = simplify(B14*B14.T)
display(Eq(MatrixSymbol('D_14', 4, 4), D14))
# Define full B matrix
B14_full = zeros(11,2) 
B14_full[2,:] = B14[0,:]
B14_full[10,:] = B14[3,:]
B14_full[6:8,:] = B14[1:3,:] 
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B14_full)

⎡\sigma_{23}__*⎤
⎢              ⎥
⎢ \sigma_{22}  ⎥
⎢              ⎥
⎢\sigma_{1'1'} ⎥
⎢              ⎥
⎣ \sigma_{23}  ⎦

      ⎡                  _________                                        ____
      ⎢                ╲╱ r_{1'2}                                    -ⅈ⋅╲╱ r_{
      ⎢                                                                       
      ⎢   _________                                       _________           
      ⎢ ╲╱ r_{1'2} ⋅(\sigma_{23} + \sigma_{23}__*)    ⅈ⋅╲╱ r_{1'2} ⋅(-\sigma_{
      ⎢ ──────────────────────────────────────────    ────────────────────────
      ⎢                     2                                               2 
B₁₄ = ⎢                                                                       
      ⎢   _________                                       _________           
      ⎢-╲╱ r_{1'2} ⋅(\sigma_{23} + \sigma_{23}__*)   -ⅈ⋅╲╱ r_{1'2} ⋅(-\sigma_{
      ⎢────────────────────────────────────────────  ─────────────────────────
      ⎢                     2                                               2 
      ⎢                                             

      ⎡           0                   \sigma_{23}__*⋅r_{1'2}               -\s
      ⎢                                                                       
      ⎢\sigma_{23}__*⋅r_{1'2}   \sigma_{23}⋅\sigma_{23}__*⋅r_{1'2}   -\sigma_{
D₁₄ = ⎢                                                                       
      ⎢-\sigma_{23}__*⋅r_{1'2}  -\sigma_{23}⋅\sigma_{23}__*⋅r_{1'2}  \sigma_{2
      ⎢                                                                       
      ⎣       2⋅r_{1'2}                 \sigma_{23}⋅r_{1'2}                 -\

igma_{23}__*⋅r_{1'2}             2⋅r_{1'2}      ⎤
                                                ⎥
23}⋅\sigma_{23}__*⋅r_{1'2}  \sigma_{23}⋅r_{1'2} ⎥
                                                ⎥
3}⋅\sigma_{23}__*⋅r_{1'2}   -\sigma_{23}⋅r_{1'2}⎥
                                                ⎥
sigma_{23}⋅r_{1'2}                   0          ⎦

In [8]:
# Substitute submatrix B_\\nu
display(Matrix([sig_31_star, sig_33, sig_22, sig_31]))
r23 = Symbol('r_{23}', real=true)
a15a = sqrt(r23)
b15a = - a15a * (sig_31_star + sig_31) / 2
c15a = - a15a * (sig_31_star - sig_31) /2
B15a = B1_real.subs(a1i, a15a).subs(b1i, b15a).subs(c1i,c15a)
display(Eq(MatrixSymbol('B_15a', 4, 2), B15a))
D15a = simplify(B15a*B15a.T)
display(Eq(MatrixSymbol('D_15a', 4, 4), D15a))
# Define full B matrix
B15a_full = zeros(11,2) 
B15a_full[3,:] = B15a[0,:]
B15a_full[9,:] = B15a[3,:]
B15a_full[5:7,:] = B15a[1:3,:] 
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B15a_full)

⎡\sigma_{31'}__*⎤
⎢               ⎥
⎢  \sigma_{33}  ⎥
⎢               ⎥
⎢  \sigma_{22}  ⎥
⎢               ⎥
⎣ \sigma_{31'}  ⎦

       ⎡                   ________                                          _
       ⎢                 ╲╱ r_{23}                                      -ⅈ⋅╲╱ 
       ⎢                                                                      
       ⎢   ________                                         ________          
       ⎢ ╲╱ r_{23} ⋅(\sigma_{31'} + \sigma_{31'}__*)    ⅈ⋅╲╱ r_{23} ⋅(-\sigma_
       ⎢ ───────────────────────────────────────────    ──────────────────────
       ⎢                      2                                               
B₁₅ₐ = ⎢                                                                      
       ⎢   ________                                         ________          
       ⎢-╲╱ r_{23} ⋅(\sigma_{31'} + \sigma_{31'}__*)   -ⅈ⋅╲╱ r_{23} ⋅(-\sigma_
       ⎢─────────────────────────────────────────────  ───────────────────────
       ⎢                      2                                               
       ⎢                                            

       ⎡           0                    \sigma_{31'}__*⋅r_{23}               -
       ⎢                                                                      
       ⎢\sigma_{31'}__*⋅r_{23}   \sigma_{31'}⋅\sigma_{31'}__*⋅r_{23}   -\sigma
D₁₅ₐ = ⎢                                                                      
       ⎢-\sigma_{31'}__*⋅r_{23}  -\sigma_{31'}⋅\sigma_{31'}__*⋅r_{23}  \sigma_
       ⎢                                                                      
       ⎣       2⋅r_{23}                  \sigma_{31'}⋅r_{23}                  

\sigma_{31'}__*⋅r_{23}               2⋅r_{23}      ⎤
                                                   ⎥
_{31'}⋅\sigma_{31'}__*⋅r_{23}  \sigma_{31'}⋅r_{23} ⎥
                                                   ⎥
{31'}⋅\sigma_{31'}__*⋅r_{23}   -\sigma_{31'}⋅r_{23}⎥
                                                   ⎥
 -\sigma_{31'}⋅r_{23}                   0          ⎦

In [9]:
# Substitute submatrix B_\\nu
display(Matrix([sig_31_star, sig_33, sig_22, sig_31]))
g = Symbol('g', real=true)
a15b = I*g*a
b15b = (sig_21_star + sig_21) / 2
c15b = (sig_21_star - sig_21) /2
d15b = -I*g*a_star
B15b = B1.subs(a1i, a15b).subs(b1i, b15b).subs(c1i,c15b).subs(d1i,d15b)
display(Eq(MatrixSymbol('B_15b', 4, 2), B15b))
D15b = simplify(B15b*B15b.T)
display(Eq(MatrixSymbol('D_15b', 4, 4), D15b))
# Define full B matrix
B15b_full = zeros(11,2) 
B15b_full[3,:] = B15b[0,:]
B15b_full[9,:] = B15b[3,:]
B15b_full[5:7,:] = B15b[1:3,:] 
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B15b_full)

⎡\sigma_{31'}__*⎤
⎢               ⎥
⎢  \sigma_{33}  ⎥
⎢               ⎥
⎢  \sigma_{22}  ⎥
⎢               ⎥
⎣ \sigma_{31'}  ⎦

        ⎡             ⅈ⋅a⋅g                                 a⋅g               
        ⎢                                                                     
        ⎢  \sigma_{21'}   \sigma_{21'}__*     ⎛  \sigma_{21'}   \sigma_{21'}__
        ⎢- ──────────── - ───────────────  -ⅈ⋅⎜- ──────────── + ──────────────
        ⎢       2                2            ⎝       2                2      
B_15b = ⎢                                                                     
        ⎢ \sigma_{21'}   \sigma_{21'}__*     ⎛  \sigma_{21'}   \sigma_{21'}__*
        ⎢ ──────────── + ───────────────   ⅈ⋅⎜- ──────────── + ───────────────
        ⎢      2                2            ⎝       2                2       
        ⎢                                                                     
        ⎣           -ⅈ⋅a__*⋅g                             a__*⋅g              

  ⎤
  ⎥
*⎞⎥
─⎟⎥
 ⎠⎥
  ⎥
⎞ ⎥
⎟ ⎥
⎠ ⎥
  ⎥
  ⎦

        ⎡                                                                     
        ⎢          0                -ⅈ⋅\sigma_{21'}__*⋅a⋅g          ⅈ⋅\sigma_{
        ⎢                                                                     
        ⎢-ⅈ⋅\sigma_{21'}__*⋅a⋅g  \sigma_{21'}⋅\sigma_{21'}__*   -\sigma_{21'}⋅
D_15b = ⎢                                                                     
        ⎢ⅈ⋅\sigma_{21'}__*⋅a⋅g   -\sigma_{21'}⋅\sigma_{21'}__*  \sigma_{21'}⋅\
        ⎢                                                                     
        ⎢               2                                                     
        ⎣     2⋅a⋅a__*⋅g             ⅈ⋅\sigma_{21'}⋅a__*⋅g         -ⅈ⋅\sigma_{

                                2      ⎤
21'}__*⋅a⋅g           2⋅a⋅a__*⋅g       ⎥
                                       ⎥
\sigma_{21'}__*  ⅈ⋅\sigma_{21'}⋅a__*⋅g ⎥
                                       ⎥
sigma_{21'}__*   -ⅈ⋅\sigma_{21'}⋅a__*⋅g⎥
                                       ⎥
 

In [10]:
# Substitute submatrix B_\\nu
display(Matrix([sig_21_star, sig_33, sig_22, sig_21]))
a16 = sqrt(r32)
b16 = a16 * (sig_21_star + sig_21) / 2
c16 = a16 * (sig_21_star - sig_21) /2
B16 = B1_real.subs(a1i, a16).subs(b1i, b16).subs(c1i,c16)
display(Eq(MatrixSymbol('B_16', 4, 2), B16))
D16 = simplify(B16*B16.T)
display(Eq(MatrixSymbol('D_16', 4, 4), D16))
# Define full B matrix
B16_full = zeros(11,2) 
B16_full[4,:] = B16[0,:]
B16_full[8,:] = B16[3,:]
B16_full[5:7,:] = B16[1:3,:] 
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B16_full)

⎡\sigma_{21'}__*⎤
⎢               ⎥
⎢  \sigma_{33}  ⎥
⎢               ⎥
⎢  \sigma_{22}  ⎥
⎢               ⎥
⎣ \sigma_{21'}  ⎦

      ⎡                   ________                                          __
      ⎢                 ╲╱ r_{32}                                      -ⅈ⋅╲╱ r
      ⎢                                                                       
      ⎢   ________                                         ________           
      ⎢-╲╱ r_{32} ⋅(\sigma_{21'} + \sigma_{21'}__*)   -ⅈ⋅╲╱ r_{32} ⋅(-\sigma_{
      ⎢─────────────────────────────────────────────  ────────────────────────
      ⎢                      2                                               2
B₁₆ = ⎢                                                                       
      ⎢   ________                                         ________           
      ⎢ ╲╱ r_{32} ⋅(\sigma_{21'} + \sigma_{21'}__*)    ⅈ⋅╲╱ r_{32} ⋅(-\sigma_{
      ⎢ ───────────────────────────────────────────    ───────────────────────
      ⎢                      2                                               2
      ⎢                                             

      ⎡           0                   -\sigma_{21'}__*⋅r_{32}                \
      ⎢                                                                       
      ⎢-\sigma_{21'}__*⋅r_{32}  \sigma_{21'}⋅\sigma_{21'}__*⋅r_{32}   -\sigma_
D₁₆ = ⎢                                                                       
      ⎢\sigma_{21'}__*⋅r_{32}   -\sigma_{21'}⋅\sigma_{21'}__*⋅r_{32}  \sigma_{
      ⎢                                                                       
      ⎣       2⋅r_{32}                  -\sigma_{21'}⋅r_{32}                  

sigma_{21'}__*⋅r_{32}               2⋅r_{32}      ⎤
                                                  ⎥
{21'}⋅\sigma_{21'}__*⋅r_{32}  -\sigma_{21'}⋅r_{32}⎥
                                                  ⎥
21'}⋅\sigma_{21'}__*⋅r_{32}   \sigma_{21'}⋅r_{32} ⎥
                                                  ⎥
\sigma_{21'}⋅r_{32}                    0          ⎦


Submatrix ${{B}_\nu}\left(\vec{A}_\nu,t\right) =
    \begin{bmatrix}
      a  \\
      -a \\
    \end{bmatrix}$ 



In [11]:
# Substitute submatrix B_\\nu
a2i, b2i = symbols('a, b', complex=True)
B2 = Matrix([[a2i], [-b2i]])
B2_real = B2.subs(b2i, a2i)
B2_complex = B2.subs(b2i, -conjugate(a2i))
D2_real = B2_real*B2_real.T;
display(simplify(D2_real))
D2_complex = B2_complex*B2_complex.T;
display(simplify(D2_complex))


⎡ 2     2⎤
⎢a    -a ⎥
⎢        ⎥
⎢  2   2 ⎥
⎣-a   a  ⎦

⎡ 2     _⎤
⎢a    a⋅a⎥
⎢        ⎥
⎢      2 ⎥
⎢  _  _  ⎥
⎣a⋅a  a  ⎦

In [12]:
display(Matrix([sig_33, sig_22]))
a21 = sqrt(r32 * sig_22 + r23 * sig_33 + I * g * (a_star * sig_23 - a * sig_23_star) - sig_21_star * sig_21 * r32 - sig_23_star * sig_23 * r32 - sig_31_star * sig_31 * r23 - sig_21*sig_21_star )
B21 = B2_real.subs(a2i, a21)
display(Eq(MatrixSymbol('B_21', 2, 1), B21))
D21 = simplify(B21*B21.T)
display(Eq(MatrixSymbol('D_21', 2, 2), D21))
# # Define full B matrix
B21_full = zeros(11,1)
B21_full[5:7,:] = B21
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B21_full)

⎡\sigma_{33}⎤
⎢           ⎥
⎣\sigma_{22}⎦

      ⎡  _____________________________________________________________________
      ⎢╲╱ -\sigma_{21'}⋅\sigma_{21'}__*⋅r_{32} - \sigma_{21'}⋅\sigma_{21'}__* 
B₂₁ = ⎢                                                                       
      ⎢   ____________________________________________________________________
      ⎣-╲╱ -\sigma_{21'}⋅\sigma_{21'}__*⋅r_{32} - \sigma_{21'}⋅\sigma_{21'}__*

______________________________________________________________________________
+ \sigma_{22}⋅r_{32} - \sigma_{23}⋅\sigma_{23}__*⋅r_{32} - \sigma_{31'}⋅\sigma
                                                                              
______________________________________________________________________________
 + \sigma_{22}⋅r_{32} - \sigma_{23}⋅\sigma_{23}__*⋅r_{32} - \sigma_{31'}⋅\sigm

______________________________________________________________________________
_{31'}__*⋅r_{23} + \sigma_{33}⋅r_{23} + ⅈ⋅g⋅(\sigma_{23}⋅a__* - \sigma_{23}__*
                                                  

      ⎡-\sigma_{21'}⋅\sigma_{21'}__*⋅r_{32} - \sigma_{21'}⋅\sigma_{21'}__* + \
D₂₁ = ⎢                                                                       
      ⎣\sigma_{21'}⋅\sigma_{21'}__*⋅r_{32} + \sigma_{21'}⋅\sigma_{21'}__* - \s

sigma_{22}⋅r_{32} - \sigma_{23}⋅\sigma_{23}__*⋅r_{32} - \sigma_{31'}⋅\sigma_{3
                                                                              
igma_{22}⋅r_{32} + \sigma_{23}⋅\sigma_{23}__*⋅r_{32} + \sigma_{31'}⋅\sigma_{31

1'}__*⋅r_{23} + \sigma_{33}⋅r_{23} + ⅈ⋅g⋅(\sigma_{23}⋅a__* - \sigma_{23}__*⋅a)
                                                                              
'}__*⋅r_{23} - \sigma_{33}⋅r_{23} - ⅈ⋅g⋅(\sigma_{23}⋅a__* - \sigma_{23}__*⋅a) 

  \sigma_{21'}⋅\sigma_{21'}__*⋅r_{32} + \sigma_{21'}⋅\sigma_{21'}__* - \sigma_
                                                                              
  -\sigma_{21'}⋅\sigma_{21'}__*⋅r_{32} - \sigma_{21'}⋅\sigma_{21'}__* + \sigma

{22}⋅r_{32} + \sigma_{23}⋅\sigma_{23}__*⋅r_{32} 

In [13]:
display(Matrix([sig_33, sig_11]))
Omega_31 = Symbol('\\Omega_{31\'}', real=true)
r31 = Symbol('r_{31\'}', real=true)
a22 = sqrt(I * Omega_31 * (sig_31_star - sig_31) + r13 * sig_33 + r31 * sig_11 - sig_31_star * sig_31 * r13)
B22 = B2_real.subs(a2i, a22)
display(Eq(MatrixSymbol('B_22', 2, 1), B22))
D22 = simplify(B22*B22.T)
display(Eq(MatrixSymbol('D_22', 2, 2), D22))
# # Define full B matrix
B22_full = zeros(11,1)
B22_full[5] = B22[0]
B22_full[7] = B22[1]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B22_full)

⎡ \sigma_{33} ⎤
⎢             ⎥
⎣\sigma_{1'1'}⎦

      ⎡  _____________________________________________________________________
      ⎢╲╱ ⅈ⋅\Omega_{31'}⋅(-\sigma_{31'} + \sigma_{31'}__*) + \sigma_{1'1'}⋅r_{
B₂₂ = ⎢                                                                       
      ⎢   ____________________________________________________________________
      ⎣-╲╱ ⅈ⋅\Omega_{31'}⋅(-\sigma_{31'} + \sigma_{31'}__*) + \sigma_{1'1'}⋅r_

__________________________________________________________________ ⎤
31'} - \sigma_{31'}⋅\sigma_{31'}__*⋅r_{1'3} + \sigma_{33}⋅r_{1'3}  ⎥
                                                                   ⎥
___________________________________________________________________⎥
{31'} - \sigma_{31'}⋅\sigma_{31'}__*⋅r_{1'3} + \sigma_{33}⋅r_{1'3} ⎦

      ⎡-ⅈ⋅\Omega_{31'}⋅(\sigma_{31'} - \sigma_{31'}__*) + \sigma_{1'1'}⋅r_{31'
D₂₂ = ⎢                                                                       
      ⎣ⅈ⋅\Omega_{31'}⋅(\sigma_{31'} - \sigma_{31'}__*) - \sigma_{1'1'}⋅r_{31'}

} - \sigma_{31'}⋅\sigma_{31'}__*⋅r_{1'3} + \sigma_{33}⋅r_{1'3}  ⅈ⋅\Omega_{31'}
                                                                              
 + \sigma_{31'}⋅\sigma_{31'}__*⋅r_{1'3} - \sigma_{33}⋅r_{1'3}   -ⅈ⋅\Omega_{31'

⋅(\sigma_{31'} - \sigma_{31'}__*) - \sigma_{1'1'}⋅r_{31'} + \sigma_{31'}⋅\sigm
                                                                              
}⋅(\sigma_{31'} - \sigma_{31'}__*) + \sigma_{1'1'}⋅r_{31'} - \sigma_{31'}⋅\sig

a_{31'}__*⋅r_{1'3} - \sigma_{33}⋅r_{1'3} ⎤
                                         ⎥
ma_{31'}__*⋅r_{1'3} + \sigma_{33}⋅r_{1'3}⎦

In [14]:
display(Matrix([sig_22, sig_11]))
r21 = Symbol('r_{21\'}', real=true)
a23 = sqrt(r12 * sig_22 + r21 * sig_11 - sig_21_star * sig_21 * r12 - sig_23_star * sig_23 * r12)
B23 = B2_real.subs(a2i, a23)
display(Eq(MatrixSymbol('B_23', 2, 1), B23))
D23 = simplify(B23*B23.T)
display(Eq(MatrixSymbol('D_i', 2, 2), D23))
# # Define full B matrix
B23_full = zeros(11,1)
B23_full[6] = B23[0]
B23_full[7] = B23[1]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B23_full)

⎡ \sigma_{22} ⎤
⎢             ⎥
⎣\sigma_{1'1'}⎦

      ⎡  _____________________________________________________________________
      ⎢╲╱ \sigma_{1'1'}⋅r_{21'} - \sigma_{21'}⋅\sigma_{21'}__*⋅r_{1'2} + \sigm
B₂₃ = ⎢                                                                       
      ⎢   ____________________________________________________________________
      ⎣-╲╱ \sigma_{1'1'}⋅r_{21'} - \sigma_{21'}⋅\sigma_{21'}__*⋅r_{1'2} + \sig

____________________________________________________ ⎤
a_{22}⋅r_{1'2} - \sigma_{23}⋅\sigma_{23}__*⋅r_{1'2}  ⎥
                                                     ⎥
_____________________________________________________⎥
ma_{22}⋅r_{1'2} - \sigma_{23}⋅\sigma_{23}__*⋅r_{1'2} ⎦

     ⎡\sigma_{1'1'}⋅r_{21'} - \sigma_{21'}⋅\sigma_{21'}__*⋅r_{1'2} + \sigma_{2
Dᵢ = ⎢                                                                        
     ⎣-\sigma_{1'1'}⋅r_{21'} + \sigma_{21'}⋅\sigma_{21'}__*⋅r_{1'2} - \sigma_{

2}⋅r_{1'2} - \sigma_{23}⋅\sigma_{23}__*⋅r_{1'2}   -\sigma_{1'1'}⋅r_{21'} + \si
                                                                              
22}⋅r_{1'2} + \sigma_{23}⋅\sigma_{23}__*⋅r_{1'2}  \sigma_{1'1'}⋅r_{21'} - \sig

gma_{21'}⋅\sigma_{21'}__*⋅r_{1'2} - \sigma_{22}⋅r_{1'2} + \sigma_{23}⋅\sigma_{
                                                                              
ma_{21'}⋅\sigma_{21'}__*⋅r_{1'2} + \sigma_{22}⋅r_{1'2} - \sigma_{23}⋅\sigma_{2

23}__*⋅r_{1'2}⎤
              ⎥
3}__*⋅r_{1'2} ⎦

In [15]:
display(Matrix([sig_23_star, sig_23]))
a24 = sqrt(-2*I*g*a_star*sig_23_star)
b24 = sqrt(2*I*g*a*sig_23)
B24 = B2.subs(a2i, a24).subs(b2i, b24)
display(Eq(MatrixSymbol('B_24', 2, 1), B24))
D24 = simplify(B24*B24.T)
display(Eq(MatrixSymbol('D_24', 2, 2), D24))
# # Define full B matrix
B24_full = zeros(11,1)
B24_full[2] = B24[0]
B24_full[10] = B24[1]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B24_full)

⎡\sigma_{23}__*⎤
⎢              ⎥
⎣ \sigma_{23}  ⎦

      ⎡     __________________________⎤
      ⎢√2⋅╲╱ -ⅈ⋅\sigma_{23}__*⋅a__*⋅g ⎥
B₂₄ = ⎢                               ⎥
      ⎢         ___________________   ⎥
      ⎣   -√2⋅╲╱ ⅈ⋅\sigma_{23}⋅a⋅g    ⎦

      ⎡                                                            ___________
      ⎢             -2⋅ⅈ⋅\sigma_{23}__*⋅a__*⋅g                -2⋅╲╱ ⅈ⋅\sigma_{
D₂₄ = ⎢                                                                       
      ⎢     ___________________   __________________________                  
      ⎣-2⋅╲╱ ⅈ⋅\sigma_{23}⋅a⋅g ⋅╲╱ -ⅈ⋅\sigma_{23}__*⋅a__*⋅g                   

________   __________________________⎤
23}⋅a⋅g ⋅╲╱ -ⅈ⋅\sigma_{23}__*⋅a__*⋅g ⎥
                                     ⎥
                                     ⎥
 2⋅ⅈ⋅\sigma_{23}⋅a⋅g                 ⎦

In [16]:
display(Matrix([sig_31_star, sig_31]))
a25 = sqrt(2*I*Omega_31*sig_31_star)
b25 = sqrt(-2*I*Omega_31*sig_31)
B25 = B2.subs(a2i, a25).subs(b2i, b25)
display(Eq(MatrixSymbol('B_25', 2, 1), B25))
D25 = simplify(B25*B25.T)
display(Eq(MatrixSymbol('D_25', 2, 2), D25))
# # Define full B matrix
B25_full = zeros(11,1)
B25_full[3] = B25[0]
B25_full[9] = B25[1]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B25_full)
display(simplify(B_full*B_full.T))

⎡\sigma_{31'}__*⎤
⎢               ⎥
⎣ \sigma_{31'}  ⎦

      ⎡     ________________________________⎤
      ⎢√2⋅╲╱ ⅈ⋅\Omega_{31'}⋅\sigma_{31'}__* ⎥
B₂₅ = ⎢                                     ⎥
      ⎢      ______________________________ ⎥
      ⎣-√2⋅╲╱ -ⅈ⋅\Omega_{31'}⋅\sigma_{31'}  ⎦

      ⎡                                                                       
      ⎢                   2⋅ⅈ⋅\Omega_{31'}⋅\sigma_{31'}__*                    
D₂₅ = ⎢                                                                       
      ⎢     ________________________________   ______________________________ 
      ⎣-2⋅╲╱ ⅈ⋅\Omega_{31'}⋅\sigma_{31'}__* ⋅╲╱ -ⅈ⋅\Omega_{31'}⋅\sigma_{31'}  

      ________________________________   ______________________________⎤
 -2⋅╲╱ ⅈ⋅\Omega_{31'}⋅\sigma_{31'}__* ⋅╲╱ -ⅈ⋅\Omega_{31'}⋅\sigma_{31'} ⎥
                                                                       ⎥
                                                                       ⎥
                     -2⋅ⅈ⋅\Omega_{31'}⋅\sigma_{31'}                    ⎦

⎡0  0                                       0                                 
⎢                                                                             
⎢0  0                                       0                                 
⎢                                                                             
⎢                                                                             
⎢0  0                          -2⋅ⅈ⋅\sigma_{23}__*⋅a__*⋅g                     
⎢                                                                             
⎢                                                                             
⎢0  0                                       0                                 
⎢                                                                             
⎢0  0                                       0                                 
⎢                                                                             
⎢0  0                            -\sigma_{23}__*⋅r_{


Submatrix ${{B}_\nu}\left(\vec{A}_\nu,t\right) =
    \begin{bmatrix}
      a  & \imu a \\
      b & -\imu c \\
      b^*  & \imu c^*  \\
      a^*  & -\imu a^*  \\
    \end{bmatrix}$ 



In [17]:
# Define submatrix B_\\nu
a3i, b3i, c3i, d3i = symbols('a, b, c, d', complex=true)
B3 = Matrix([[a3i, I*a3i], [b3i, -I*b3i], [c3i, I*c3i], [d3i, -I*d3i]])
B3_complex = B3.subs(d3i, conjugate(a3i))
Bnu = MatrixSymbol('B_\\nu', 4, 2)
Eq_Bnu = Eq(Bnu, B3_complex)
display(Eq_Bnu)
# Calc submatrix D_\\nu
D3_complex = B3_complex*B3_complex.T
Dnu = MatrixSymbol('D_\\nu', 4, 4)
Eq_Dnu = Eq(Dnu, D3_complex)
display(Eq_Dnu)

        ⎡a  ⅈ⋅a ⎤
        ⎢       ⎥
        ⎢b  -ⅈ⋅b⎥
        ⎢       ⎥
B_\nu = ⎢c  ⅈ⋅c ⎥
        ⎢       ⎥
        ⎢_     _⎥
        ⎣a  -ⅈ⋅a⎦

        ⎡                         _⎤
        ⎢  0    2⋅a⋅b    0    2⋅a⋅a⎥
        ⎢                          ⎥
        ⎢2⋅a⋅b    0    2⋅b⋅c    0  ⎥
        ⎢                          ⎥
D_\nu = ⎢                         _⎥
        ⎢  0    2⋅b⋅c    0    2⋅c⋅a⎥
        ⎢                          ⎥
        ⎢    _             _       ⎥
        ⎣2⋅a⋅a    0    2⋅c⋅a    0  ⎦

In [18]:
# Substitute submatrix B_\\nu
display(Matrix([sig_23_star, sig_31_star, sig_31, sig_23]))
a31 = I*g*a_star/2
b31 = sig_31_star
c31 = sig_31
d31 = -I*g*a/2
B31 = B3.subs(a3i, a31).subs(b3i, b31).subs(c3i,c31).subs(d3i,d31)
display(Eq(MatrixSymbol('B_31', 4, 2), B31))
D31 = simplify(B31*B31.T)
display(Eq(MatrixSymbol('D_31', 4, 4), D31))
# Define full B matrix
B31_full = zeros(11,2)
B31_full[2:4,:] = B31[0:2,:]
B31_full[9:11,:] = B31[2:4,:]
display(B31_full)
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B31_full)

⎡\sigma_{23}__* ⎤
⎢               ⎥
⎢\sigma_{31'}__*⎥
⎢               ⎥
⎢ \sigma_{31'}  ⎥
⎢               ⎥
⎣  \sigma_{23}  ⎦

      ⎡   ⅈ⋅a__*⋅g           -a__*⋅g      ⎤
      ⎢   ────────           ────────     ⎥
      ⎢      2                  2         ⎥
      ⎢                                   ⎥
      ⎢\sigma_{31'}__*  -ⅈ⋅\sigma_{31'}__*⎥
B₃₁ = ⎢                                   ⎥
      ⎢ \sigma_{31'}      ⅈ⋅\sigma_{31'}  ⎥
      ⎢                                   ⎥
      ⎢    -ⅈ⋅a⋅g             -a⋅g        ⎥
      ⎢    ───────            ─────       ⎥
      ⎣       2                 2         ⎦

      ⎡                                                                       
      ⎢                                                                       
      ⎢           0                 ⅈ⋅\sigma_{31'}__*⋅a__*⋅g                  
      ⎢                                                                       
      ⎢                                                                       
      ⎢ⅈ⋅\sigma_{31'}__*⋅a__*⋅g                0                 2⋅\sigma_{31'
D₃₁ = ⎢                                                                       
      ⎢           0              2⋅\sigma_{31'}⋅\sigma_{31'}__*               
      ⎢                                                                       
      ⎢               2                                                       
      ⎢       a⋅a__*⋅g                                                        
      ⎢       ─────────                        0                      -ⅈ⋅\sigm
      ⎣           2                                 

⎡       0                 0         ⎤
⎢                                   ⎥
⎢       0                 0         ⎥
⎢                                   ⎥
⎢   ⅈ⋅a__*⋅g           -a__*⋅g      ⎥
⎢   ────────           ────────     ⎥
⎢      2                  2         ⎥
⎢                                   ⎥
⎢\sigma_{31'}__*  -ⅈ⋅\sigma_{31'}__*⎥
⎢                                   ⎥
⎢       0                 0         ⎥
⎢                                   ⎥
⎢       0                 0         ⎥
⎢                                   ⎥
⎢       0                 0         ⎥
⎢                                   ⎥
⎢       0                 0         ⎥
⎢                                   ⎥
⎢       0                 0         ⎥
⎢                                   ⎥
⎢ \sigma_{31'}      ⅈ⋅\sigma_{31'}  ⎥
⎢                                   ⎥
⎢    -ⅈ⋅a⋅g             -a⋅g        ⎥
⎢    ───────            ─────       ⎥
⎣       2                 2         ⎦

In [19]:
# Substitute submatrix B_\\nu
display(Matrix([sig_23_star, sig_21_star, sig_21, sig_23]))
a32 = -I*g*a_star/2
b32 = sig_21_star
c32 = sig_21
d32 = I*g*a/2
B32 = B3.subs(a3i, a32).subs(b3i, b32).subs(c3i,c32).subs(d3i,d32)
display(Eq(MatrixSymbol('B_32', 4, 2), B32))
D32 = simplify(B32*B32.T)
display(Eq(MatrixSymbol('D_32', 4, 4), D32))
# Define full B matrix
B32_full = zeros(11,2)
B32_full[2,:] = B32[0,:]
B32_full[4,:] = B32[1,:]
B32_full[8,:] = B32[2,:]
B32_full[10,:] = B32[3,:]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B32_full)

⎡\sigma_{23}__* ⎤
⎢               ⎥
⎢\sigma_{21'}__*⎥
⎢               ⎥
⎢ \sigma_{21'}  ⎥
⎢               ⎥
⎣  \sigma_{23}  ⎦

      ⎡  -ⅈ⋅a__*⋅g            a__*⋅g      ⎤
      ⎢  ──────────           ──────      ⎥
      ⎢      2                  2         ⎥
      ⎢                                   ⎥
      ⎢\sigma_{21'}__*  -ⅈ⋅\sigma_{21'}__*⎥
B₃₂ = ⎢                                   ⎥
      ⎢ \sigma_{21'}      ⅈ⋅\sigma_{21'}  ⎥
      ⎢                                   ⎥
      ⎢     ⅈ⋅a⋅g              a⋅g        ⎥
      ⎢     ─────              ───        ⎥
      ⎣       2                 2         ⎦

      ⎡                                                                       
      ⎢                                                                       
      ⎢            0                -ⅈ⋅\sigma_{21'}__*⋅a__*⋅g                 
      ⎢                                                                       
      ⎢                                                                       
      ⎢-ⅈ⋅\sigma_{21'}__*⋅a__*⋅g                0                 2⋅\sigma_{21
D₃₂ = ⎢                                                                       
      ⎢            0              2⋅\sigma_{21'}⋅\sigma_{21'}__*              
      ⎢                                                                       
      ⎢                2                                                      
      ⎢        a⋅a__*⋅g                                                       
      ⎢        ─────────                        0                       ⅈ⋅\sig
      ⎣            2                                

In [20]:
# Substitute submatrix B_\\nu
display(Matrix([sig_23_star, sig_21, sig_21_star, sig_23]))
gamma23, gamma12, gamma13 = symbols('\\gamma_{23}, \\gamma_{1\'2}, \\gamma_{1\'3}', real=true)
a33 = sqrt((gamma23 + gamma12 - gamma13)/2)
b33 = a33 * sig_31
c33 = a33 * sig_31_star
d33 = a33
B33 = B3.subs(a3i, a33).subs(b3i, b33).subs(c3i,c33).subs(d3i,d33)
display(Eq(MatrixSymbol('B_33', 4, 2), B33))
D33 = simplify(B33*B33.T)
display(Eq(MatrixSymbol('D_33', 4, 4), D33))
# Define full B matrix
B33_full = zeros(11,2)
B33_full[2,:] = B33[0,:]
B33_full[8,:] = B33[1,:]
B33_full[4,:] = B33[2,:]
B33_full[10,:] = B33[3,:]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B33_full)
display(simplify(B_full*B_full.T))

⎡\sigma_{23}__* ⎤
⎢               ⎥
⎢ \sigma_{21'}  ⎥
⎢               ⎥
⎢\sigma_{21'}__*⎥
⎢               ⎥
⎣  \sigma_{23}  ⎦

      ⎡            ___________________________________________                
      ⎢           ╱ \gamma_{1'2}   \gamma_{1'3}   \gamma_{23}                 
      ⎢          ╱  ──────────── - ──────────── + ───────────                 
      ⎢        ╲╱        2              2              2                      
      ⎢                                                                       
      ⎢                  ___________________________________________          
      ⎢                 ╱ \gamma_{1'2}   \gamma_{1'3}   \gamma_{23}           
      ⎢ \sigma_{31'}⋅  ╱  ──────────── - ──────────── + ───────────      -ⅈ⋅\s
      ⎢              ╲╱        2              2              2                
B₃₃ = ⎢                                                                       
      ⎢                    ___________________________________________        
      ⎢                   ╱ \gamma_{1'2}   \gamma_{1'3}   \gamma_{23}         
      ⎢\sigma_{31'}__*⋅  ╱  ──────────── - ─────────

      ⎡                           0                                      \sigm
      ⎢                                                                       
      ⎢\sigma_{31'}⋅(\gamma_{1'2} - \gamma_{1'3} + \gamma_{23})               
D₃₃ = ⎢                                                                       
      ⎢                           0                              \sigma_{31'}⋅
      ⎢                                                                       
      ⎣       \gamma_{1'2} - \gamma_{1'3} + \gamma_{23}                       

a_{31'}⋅(\gamma_{1'2} - \gamma_{1'3} + \gamma_{23})                           
                                                                              
                      0                                      \sigma_{31'}⋅\sig
                                                                              
\sigma_{31'}__*⋅(\gamma_{1'2} - \gamma_{1'3} + \gamma_{23})                   
                                                   

⎡0  0                                                                   0     
⎢                                                                             
⎢0  0                                                                   0     
⎢                                                                             
⎢                                                                             
⎢0  0                                                      -2⋅ⅈ⋅\sigma_{23}__*
⎢                                                                             
⎢                                                                             
⎢0  0                                                       ⅈ⋅\sigma_{31'}__*⋅
⎢                                                                             
⎢0  0                                                       -ⅈ⋅\sigma_{21'}__*
⎢                                                                             
⎢0  0                                               


Submatrix ${{B}_\nu}\left(\vec{A}_\nu,t\right) =
    \begin{bmatrix}
      a  & \imu a \\
      a & -\imu a \\
    \end{bmatrix}$ 



In [21]:
# Define submatrix B_\\nu
a4i = symbols('a', complex=true)
B4 = Matrix([[a4i, I*a4i], [a4i, -I*a4i]])
Bnu = MatrixSymbol('B_\\nu', 2, 2)
Eq_Bnu = Eq(Bnu, B4)
display(Eq_Bnu)
# Calc submatrix D_\\nu
D4 = B4*B4.T
Dnu = MatrixSymbol('D_\\nu', 2, 2)
Eq_Dnu = Eq(Dnu, D4)
display(Eq_Dnu)

        ⎡a  ⅈ⋅a ⎤
B_\nu = ⎢       ⎥
        ⎣a  -ⅈ⋅a⎦

        ⎡         2⎤
        ⎢ 0    2⋅a ⎥
D_\nu = ⎢          ⎥
        ⎢   2      ⎥
        ⎣2⋅a    0  ⎦

In [22]:
# Substitute submatrix B_\\nu
display(Matrix([sig_23_star, sig_23]))
a41 = sqrt(((2*gamma23 - r23 - r13) * sig_33 + r32* sig_22 - gamma12 + gamma13 - gamma23 - a* a_star * g**2 - 2 * r12 - 2 * r32 + 2 * sqrt(I* sig_23 * a * g) * sqrt(-I*sig_23_star *a_star * g)) / 2) 
B41 = B4.subs(a4i, a41)
display(Eq(MatrixSymbol('B_41', 2, 2), B41))
D41 = simplify(B41*B41.T)
display(Eq(MatrixSymbol('D_41', 2, 2), D41))
# Define full B matrix
B41_full = zeros(11,2)
B41_full[2,:] = B41[0,:]
B41_full[10,:] = B41[1,:]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1],B41_full)


⎡\sigma_{23}__*⎤
⎢              ⎥
⎣ \sigma_{23}  ⎦

      ⎡     __________________________________________________________________
      ⎢    ╱                                                                  
      ⎢   ╱    \gamma_{1'2}   \gamma_{1'3}   \gamma_{23}   \sigma_{22}⋅r_{32} 
      ⎢  ╱   - ──────────── + ──────────── - ─────────── + ────────────────── 
      ⎢╲╱           2              2              2                2          
B₄₁ = ⎢                                                                       
      ⎢     __________________________________________________________________
      ⎢    ╱                                                                  
      ⎢   ╱    \gamma_{1'2}   \gamma_{1'3}   \gamma_{23}   \sigma_{22}⋅r_{32} 
      ⎢  ╱   - ──────────── + ──────────── - ─────────── + ────────────────── 
      ⎣╲╱           2              2              2                2          

______________________________________________________________________________
                                                   

      ⎡                                                                       
      ⎢                                                                       
D₄₁ = ⎢                                                                       
      ⎢                                                                       
      ⎣-\gamma_{1'2} + \gamma_{1'3} - \gamma_{23} + \sigma_{22}⋅r_{32} - \sigm

                                                                              
                              0                                               
                                                                              
                                                     2                        
a_{33}⋅(-2⋅\gamma_{23} + r_{1'3} + r_{23}) - a⋅a__*⋅g  - 2⋅r_{1'2} - 2⋅r_{32} 

                                                                              
                                                        -\gamma_{1'2} + \gamma
                                                  

In [23]:
# Substitute submatrix B_\\nu
display(Matrix([sig_31_star, sig_31]))
a42 = sqrt(((2*gamma13 - r21 - r31) * sig_11 + r12* sig_22 + r13* sig_33 - 2 * sig_31 * sig_31_star - 2 * a* a_star * g**2 - 2 * r13 - 2 * r23 + 2 * sqrt(I* Omega_31 * sig_31_star) * sqrt(-I* Omega_31 * sig_31)) / 2) 
B42 = B4.subs(a4i, a42)
display(Eq(MatrixSymbol('B_42', 2, 2), B42))
D42 = simplify(B42*B42.T)
display(Eq(MatrixSymbol('D_42', 2, 2), D42))
# Define full B matrix
B42_full = zeros(11,2)
B42_full[3,:] = B42[0,:]
B42_full[9,:] = B42[1,:]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1], B42_full)

⎡\sigma_{31'}__*⎤
⎢               ⎥
⎣ \sigma_{31'}  ⎦

      ⎡    ___________________________________________________________________
      ⎢   ╱ \sigma_{1'1'}⋅(2⋅\gamma_{1'3} - r_{21'} - r_{31'})   \sigma_{22}⋅r
      ⎢  ╱  ────────────────────────────────────────────────── + ─────────────
      ⎢╲╱                           2                                     2   
B₄₂ = ⎢                                                                       
      ⎢    ___________________________________________________________________
      ⎢   ╱ \sigma_{1'1'}⋅(2⋅\gamma_{1'3} - r_{21'} - r_{31'})   \sigma_{22}⋅r
      ⎢  ╱  ────────────────────────────────────────────────── + ─────────────
      ⎣╲╱                           2                                     2   

______________________________________________________________________________
_{1'2}                                  \sigma_{33}⋅r_{1'3}           2       
────── - \sigma_{31'}⋅\sigma_{31'}__* + ─────────────────── - a⋅a__*⋅g  - r_{1
                                                 2 

      ⎡                                                                       
      ⎢                                                                       
D₄₂ = ⎢                                                                       
      ⎢                                                                       
      ⎣-\sigma_{1'1'}⋅(-2⋅\gamma_{1'3} + r_{21'} + r_{31'}) + \sigma_{22}⋅r_{1

                                                                              
                                               0                              
                                                                              
                                                                       2      
'2} - 2⋅\sigma_{31'}⋅\sigma_{31'}__* + \sigma_{33}⋅r_{1'3} - 2⋅a⋅a__*⋅g  - 2⋅r

                                                                              
                                                                              
                                                  

In [24]:
# Substitute submatrix B_\\nu
display(Matrix([sig_21_star, sig_21]))
a43= sqrt(((2*gamma12 - r21 - r31) * sig_11 + r12* sig_22 + r13* sig_33 - 2 * sig_21 * sig_21_star  - 2 * r12 - 2 * r32 - sig_31_star * sig_31 * (gamma12 - gamma13 + gamma23)) / 2) 
B43 = B4.subs(a4i, a43)
display(Eq(MatrixSymbol('B_43', 2, 2), B43))
D43 = simplify(B43*B43.T)
display(Eq(MatrixSymbol('D_43', 2, 2), D43))
# Define full B matrix
B43_full = zeros(11,2)
B43_full[4,:] = B43[0,:]
B43_full[8,:] = B43[1,:]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1], B43_full)


⎡\sigma_{21'}__*⎤
⎢               ⎥
⎣ \sigma_{21'}  ⎦

      ⎡    ___________________________________________________________________
      ⎢   ╱ \sigma_{1'1'}⋅(2⋅\gamma_{1'2} - r_{21'} - r_{31'})                
      ⎢  ╱  ────────────────────────────────────────────────── - \sigma_{21'}⋅
      ⎢╲╱                           2                                         
B₄₃ = ⎢                                                                       
      ⎢    ___________________________________________________________________
      ⎢   ╱ \sigma_{1'1'}⋅(2⋅\gamma_{1'2} - r_{21'} - r_{31'})                
      ⎢  ╱  ────────────────────────────────────────────────── - \sigma_{21'}⋅
      ⎣╲╱                           2                                         

______________________________________________________________________________
                  \sigma_{22}⋅r_{1'2}   \sigma_{31'}⋅\sigma_{31'}__*⋅(\gamma_{
\sigma_{21'}__* + ─────────────────── - ──────────────────────────────────────
                           2                       

      ⎡                                                                       
D₄₃ = ⎢                                                                       
      ⎣-\sigma_{1'1'}⋅(-2⋅\gamma_{1'2} + r_{21'} + r_{31'}) - 2⋅\sigma_{21'}⋅\

                                          0                                   
                                                                              
sigma_{21'}__* + \sigma_{22}⋅r_{1'2} - \sigma_{31'}⋅\sigma_{31'}__*⋅(\gamma_{1

                                                                              
                                                                              
'2} - \gamma_{1'3} + \gamma_{23}) + \sigma_{33}⋅r_{1'3} - 2⋅r_{1'2} - 2⋅r_{32}

  -\sigma_{1'1'}⋅(-2⋅\gamma_{1'2} + r_{21'} + r_{31'}) - 2⋅\sigma_{21'}⋅\sigma
                                                                              
                                                                              

_{21'}__* + \sigma_{22}⋅r_{1'2} - \sigma_{31'}⋅\

In [25]:
# Substitute submatrix B_\\nu
display(Matrix([a_star, a]))
nth, kappa = symbols('n_\\mathrm{th}, \\kappa', real = true)
a44=  sqrt(kappa * nth /2)
B44 = B4.subs(a4i, a44)
display(Eq(MatrixSymbol('B_44', 2, 2), B44))
D44 = simplify(B44*B44.T)
display(Eq(MatrixSymbol('D_44', 2, 2), D44))
# Define full B matrix
B44_full = zeros(11,2)
B44_full[0,:] = B44[0,:]
B44_full[1,:] = B44[1,:]
size_B= B_full.shape
B_full = B_full.col_insert(size_B[1], B44_full)

⎡a__*⎤
⎢    ⎥
⎣ a  ⎦

      ⎡     ______________________          ______________________ ⎤
      ⎢√2⋅╲╱ \kappa⋅n_\mathrm{th}    √2⋅ⅈ⋅╲╱ \kappa⋅n_\mathrm{th}  ⎥
      ⎢───────────────────────────   ───────────────────────────── ⎥
      ⎢             2                              2               ⎥
B₄₄ = ⎢                                                            ⎥
      ⎢     ______________________          ______________________ ⎥
      ⎢√2⋅╲╱ \kappa⋅n_\mathrm{th}   -√2⋅ⅈ⋅╲╱ \kappa⋅n_\mathrm{th}  ⎥
      ⎢───────────────────────────  ───────────────────────────────⎥
      ⎣             2                              2               ⎦

      ⎡         0            \kappa⋅n_\mathrm{th}⎤
D₄₄ = ⎢                                          ⎥
      ⎣\kappa⋅n_\mathrm{th}           0          ⎦

In [26]:
display(Eq(MatrixSymbol('D', 11, 11), simplify(B_full*B_full.T)))
size_B= B_full.shape
display(Eq(MatrixSymbol('B', 11, size_B[1]), B_full))

    ⎡         0            \kappa⋅n_\mathrm{th}                               
    ⎢                                                                         
    ⎢\kappa⋅n_\mathrm{th}           0                                         
    ⎢                                                                         
    ⎢         0                     0                                 -2⋅ⅈ⋅\si
    ⎢                                                                         
    ⎢         0                     0                                  ⅈ⋅\sigm
    ⎢                                                                         
    ⎢         0                     0                                 -ⅈ⋅\sigm
    ⎢                                                                         
D = ⎢         0                     0                                   -\sigm
    ⎢                                                                         
    ⎢         0                     0               

    ⎡                                                                         
    ⎢                                                                         
    ⎢                     0                                             0     
    ⎢                                                                         
    ⎢                                                                         
    ⎢                                                                         
    ⎢                                                                         
    ⎢                     0                                             0     
    ⎢                                                                         
    ⎢                                                                         
    ⎢                                                                         
    ⎢                                                                         
    ⎢                  ________                     

In [27]:
display(B44_full)

⎡     ______________________          ______________________ ⎤
⎢√2⋅╲╱ \kappa⋅n_\mathrm{th}    √2⋅ⅈ⋅╲╱ \kappa⋅n_\mathrm{th}  ⎥
⎢───────────────────────────   ───────────────────────────── ⎥
⎢             2                              2               ⎥
⎢                                                            ⎥
⎢     ______________________          ______________________ ⎥
⎢√2⋅╲╱ \kappa⋅n_\mathrm{th}   -√2⋅ⅈ⋅╲╱ \kappa⋅n_\mathrm{th}  ⎥
⎢───────────────────────────  ───────────────────────────────⎥
⎢             2                              2               ⎥
⎢                                                            ⎥
⎢             0                              0               ⎥
⎢                                                            ⎥
⎢             0                              0               ⎥
⎢                                                            ⎥
⎢             0                              0               ⎥
⎢                                                      